## Problem 15: "Advanced" Pandemic Flu Spread

Project 3 considered a (trivial) discrete-event simulation of pandemic flu spread in a classroom. (Project 4 concerned a continuous-time, deterministic simulation that doesn’t apply here.) If you feel a little more adventuresome, I’d like you to think about a bigger-and-better simulation involving a larger population. Here’s a potential scenario (there are many other interesting ones — feel free to be imaginative):

- Some infectious people enter a population of susceptibles, and some of the susceptibles become infected.
- There is a short period of a couple of days before a newly-infected person in turn becomes infectious.
- When a person recovers (or dies), the person is not again susceptible.
- Infectiousness or death can be mitigated by masking, social distancing, etc.
- Infectiousness or death can be mitigated by vaccination. Vaccines can be delivered in one or two doses. But there could be supply chain issues.
- Even if a vaccine requires two doses, the vaccine nevertheless provides partial immunization after even only one dose. Might you immediately give everyone only one dose instead of two, and hope that the supply chain catches up so that you can “eventually” give everyone two doses?

To determine whether a particular strategy is any good, you probably ought to consider the number of people who eventually get infected (or die), the length of the epidemic, etc.


In [1]:
%load_ext autoreload
%autoreload 2

In [2]:
import itertools
import os
import shutil
import sys

import pandas as pd

In [3]:
# Get the current working directory (where the notebook is located)
current_dir = os.getcwd()

# Construct the path to the src folder
src_path = os.path.join(current_dir, "src")

# Add src_path to sys.path
sys.path.insert(0, src_path)

In [4]:
def clear_or_create_directory(directory_path):
    """
    Clears all contents of the given directory if it exists.
    If it doesn't exist, creates the directory.
    """
    try:
        if os.path.exists(directory_path):
            print(f"Directory '{directory_path}' exists. Clearing contents...")
            for filename in os.listdir(directory_path):
                file_path = os.path.join(directory_path, filename)
                # Handle files, symlinks, and directories appropriately
                if os.path.isfile(file_path) or os.path.islink(file_path):
                    os.unlink(file_path)  # Delete file or symlink
                elif os.path.isdir(file_path):
                    shutil.rmtree(file_path)  # Recursively delete directory
            print(f"All contents of '{directory_path}' have been cleared.")
        else:
            print(f"Directory '{directory_path}' does not exist. Creating it...")
            os.makedirs(directory_path)  # Create the directory
            print(f"Directory '{directory_path}' has been created.")
    except Exception as e:
        print(f"Error processing directory '{directory_path}': {e}")


results_dir = os.path.join(current_dir, "results")
clear_or_create_directory(results_dir)

Directory 'c:\Users\louis\OneDrive - Georgia Institute of Technology\GATech\2024 sem 4 fall\ISYE 6644\Project\pandemic-flu-spread\pandemic-flu-spread\results' exists. Clearing contents...
All contents of 'c:\Users\louis\OneDrive - Georgia Institute of Technology\GATech\2024 sem 4 fall\ISYE 6644\Project\pandemic-flu-spread\pandemic-flu-spread\results' have been cleared.


In [5]:
from functools import partial

from src.batch_run import SimulationBatchRunner
from src.policies import (
    mask_policy,
    social_distancing_policy,
    supply_constrained_vaccination_policy,
)

In [6]:
infection_rates = [0.4, 0.5, 0.6, 0.7, 0.8]
mask_start_days = [5, 10, 15]
social_distancing_start_days = [5, 10, 15]

# Initialize empty DataFrames to store results
histogram_results_df = pd.DataFrame()
state_over_time_results_df = pd.DataFrame()
loading_tracker = 0

for infection_rate, mask_start_day, social_distancing_start_day in itertools.product(
    infection_rates,
    mask_start_days,
    social_distancing_start_days,
):
    print(
        f"Running simulation with infection_rate={infection_rate}, mask_start_day={mask_start_day}, social_distancing_start_day={social_distancing_start_day}"
    )
    print("--------------------------------------------------")
    percent_complete = (
        loading_tracker
        / (
            len(infection_rates)
            * len(mask_start_days)
            * len(social_distancing_start_days)
        )
        * 100
    )
    print(f"Percent complete: {percent_complete:.2f}%")
    loading_tracker += 1
    # Define the policies
    MASK_POLICY = partial(
        mask_policy, start_day=mask_start_day, max_prob_mask=0.8, growth_rate=0.05
    )
    SOCIAL_DIST_POLICY = partial(
        social_distancing_policy, start_day=social_distancing_start_day
    )

    VAC_POLICY = partial(
        supply_constrained_vaccination_policy,
        initial_supply_rate=0.01,
        max_supply_rate=0.15,
        growth_type="logistic",
    )

    # Define simulation parameters
    simulation_params = {
        "population_size": 10_000,  # Total number of individuals in the simulation.
        "initial_infected": 0.01,  # Initial infected individuals in the simulation.
        # If an integer, this specifies the absolute number of infected individuals at the start.
        # If a float between 0 and 1, it represents the fraction of the population that is initially infected.
        "infection_rate": infection_rate,  # Base probability of infection per contact (0 <= infection_rate <= 1).
        # Higher values mean more likely transmission on each contact.
        "incubation_period": 7,  # Number of days from infection until an individual becomes infectious.
        # This delay represents the period during which the individual is infected but not yet able to spread the disease.
        "infectious_period": 14,  # Number of days an individual remains infectious once they become infectious.
        # After this period, the individual either recovers or dies.
        "base_contacts": 10,  # Average number of daily contacts for individuals who are not social distancing.
        # This is the number of interactions per day where infection could potentially spread.
        "social_distancing_rate": 0.3,  # Factor to reduce the number of contacts for individuals practicing social distancing (0 <= social_distancing_rate <= 1).
        # A rate of 0.3 means social distancing reduces contacts to 30% of the base contacts.
        "mortality_rate": 0.01,  # Probability of dying from the disease for infected individuals (0 <= mortality_rate <= 1).
        # A rate of 0.01 implies a 1% chance of death for each infected individual.
        "mask_effectiveness": 0.5,  # Effectiveness of masks in reducing infection probability.
        # If an individual wears a mask, the infection rate for that contact is multiplied by this factor.
        # For example, a value of 0.5 means masks reduce the infection risk by 50%.
        "partial_vaccine_effectiveness": 0.5,  # Effectiveness of a single vaccine dose in reducing infection probability.
        # The infection rate is multiplied by this factor for individuals with partial vaccination.
        # For example, 0.5 means partial vaccination reduces the infection risk by 50%.
        "full_vaccine_effectiveness": 0,  # Effectiveness of full vaccination (two or more doses) in reducing infection probability.
        # The infection rate is multiplied by this factor for fully vaccinated individuals.
        # A value of 0 implies complete immunity, with no risk of infection after full vaccination.
        "mask_policy": MASK_POLICY,  # Function that defines the mask-wearing policy for individuals.
        # This function is invoked daily to decide whether each individual should wear a mask.
        "dist_policy": SOCIAL_DIST_POLICY,  # Function that defines the social distancing policy for individuals.
        # This function is invoked daily to determine each individual’s social distancing behavior.
        "vac_policy": VAC_POLICY,  # Function that defines the vaccination policy for the simulation.
        # This function is called daily to decide which individuals receive vaccine doses.
        "random_seed": 42,  # Seed for the random number generator, ensuring reproducibility of the simulation.
        # Setting this seed allows the simulation to produce the same results on repeated runs.
    }

    # Number of simulation runs
    replications = 100

    # Create the SimulationBatchRunner
    batch_runner = SimulationBatchRunner(
        replications=replications,
        simulation_params=simulation_params,
    )

    # Run all simulations
    batch_runner.run()

    # Define the path to save the results
    hist_save_path = os.path.join(
        results_dir,
        f"ir{infection_rate}_ms{mask_start_day}_sds{social_distancing_start_day}_hist.png",
    )

    state_save_path = os.path.join(
        results_dir,
        f"ir{infection_rate}_ms{mask_start_day}_sds{social_distancing_start_day}_state.png",
    )

    # Titles for the plots
    title_hist = (
        f"Statistical Summary\n"
        f"Infection Rate: {infection_rate}\n"
        f"Mask Start Day: {mask_start_day}\n"
        f"Social Distancing Start Day: {social_distancing_start_day}"
    )

    title_state = (
        f"State Over Time\n"
        f"Infection Rate: {infection_rate}\n"
        f"Mask Start Day: {mask_start_day}\n"
        f"Social Distancing Start Day: {social_distancing_start_day}"
    )

    # Plot histograms with statistical summaries and save results
    hist_df = batch_runner.plot_histograms(
        save_path=hist_save_path,
        print_graphs=False,
        title=title_hist,
    )
    hist_df = hist_df.assign(
        infection_rate=infection_rate,
        mask_start_day=mask_start_day,
        social_distancing_start_day=social_distancing_start_day,
    )
    histogram_results_df = pd.concat(
        [histogram_results_df, hist_df],
        ignore_index=True,
    )

    # Plot the expected counts over time and save results
    state_df = batch_runner.plot_state_over_time(
        save_path=state_save_path,
        print_graphs=False,
        title=title_state,
    )
    state_df = state_df.assign(
        infection_rate=infection_rate,
        mask_start_day=mask_start_day,
        social_distancing_start_day=social_distancing_start_day,
    )
    state_over_time_results_df = pd.concat(
        [state_over_time_results_df, state_df],
        ignore_index=True,
    )

INFO:src.batch_run:Starting batch run of simulations.


Running simulation with infection_rate=0.4, mask_start_day=5, social_distancing_start_day=5
--------------------------------------------------
Percent complete: 0.00%


INFO:src.batch_run:All simulations completed. Processing results...
INFO:src.batch_run:Results aggregation completed.


Histogram saved to c:\Users\louis\OneDrive - Georgia Institute of Technology\GATech\2024 sem 4 fall\ISYE 6644\Project\pandemic-flu-spread\pandemic-flu-spread\results\ir0.4_ms5_sds5_hist.png


INFO:src.batch_run:Starting batch run of simulations.


...

Histogram saved to c:\Users\louis\OneDrive - Georgia Institute of Technology\GATech\2024 sem 4 fall\ISYE 6644\Project\pandemic-flu-spread\pandemic-flu-spread\results\ir0.8_ms15_sds10_hist.png


INFO:src.batch_run:Starting batch run of simulations.


Plot saved to c:\Users\louis\OneDrive - Georgia Institute of Technology\GATech\2024 sem 4 fall\ISYE 6644\Project\pandemic-flu-spread\pandemic-flu-spread\results\ir0.8_ms15_sds10_state.png
Running simulation with infection_rate=0.8, mask_start_day=15, social_distancing_start_day=15
--------------------------------------------------
Percent complete: 97.78%


INFO:src.batch_run:All simulations completed. Processing results...
INFO:src.batch_run:Results aggregation completed.


Histogram saved to c:\Users\louis\OneDrive - Georgia Institute of Technology\GATech\2024 sem 4 fall\ISYE 6644\Project\pandemic-flu-spread\pandemic-flu-spread\results\ir0.8_ms15_sds15_hist.png
Plot saved to c:\Users\louis\OneDrive - Georgia Institute of Technology\GATech\2024 sem 4 fall\ISYE 6644\Project\pandemic-flu-spread\pandemic-flu-spread\results\ir0.8_ms15_sds15_state.png


In [7]:
# Save the DataFrames to Excel files
histogram_results_df.to_excel(
    os.path.join(results_dir, "histogram_results.xlsx"),
    index=False,
)

state_over_time_results_df.to_excel(
    os.path.join(results_dir, "state_over_time_results.xlsx"),
    index=False,
)

In [8]:

# Run simulation when mask_start_day and social_distancing_start_day are are not enforced (i.e., set to a large number 9999 days) and vaccination supply is 0
# Define the policies
MASK_POLICY = partial(
    mask_policy, start_day=9999, max_prob_mask=0.8, growth_rate=0.05
)
SOCIAL_DIST_POLICY = partial(
    social_distancing_policy, start_day=9999
)

VAC_POLICY = partial(
    supply_constrained_vaccination_policy,
    initial_supply_rate=0,
    max_supply_rate=0,
    growth_type="logistic",
)

# Define simulation parameters
simulation_params = {
    "population_size": 10_000,  # Total number of individuals in the simulation.
    "initial_infected": 0.01,  # Initial infected individuals in the simulation.
    # If an integer, this specifies the absolute number of infected individuals at the start.
    # If a float between 0 and 1, it represents the fraction of the population that is initially infected.
    "infection_rate": 0.4,  # Base probability of infection per contact (0 <= infection_rate <= 1).
    # Higher values mean more likely transmission on each contact.
    "incubation_period": 7,  # Number of days from infection until an individual becomes infectious.
    # This delay represents the period during which the individual is infected but not yet able to spread the disease.
    "infectious_period": 14,  # Number of days an individual remains infectious once they become infectious.
    # After this period, the individual either recovers or dies.
    "base_contacts": 10,  # Average number of daily contacts for individuals who are not social distancing.
    # This is the number of interactions per day where infection could potentially spread.
    "social_distancing_rate": 0.3,  # Factor to reduce the number of contacts for individuals practicing social distancing (0 <= social_distancing_rate <= 1).
    # A rate of 0.3 means social distancing reduces contacts to 30% of the base contacts.
    "mortality_rate": 0.01,  # Probability of dying from the disease for infected individuals (0 <= mortality_rate <= 1).
    # A rate of 0.01 implies a 1% chance of death for each infected individual.
    "mask_effectiveness": 0.5,  # Effectiveness of masks in reducing infection probability.
    # If an individual wears a mask, the infection rate for that contact is multiplied by this factor.
    # For example, a value of 0.5 means masks reduce the infection risk by 50%.
    "partial_vaccine_effectiveness": 0.5,  # Effectiveness of a single vaccine dose in reducing infection probability.
    # The infection rate is multiplied by this factor for individuals with partial vaccination.
    # For example, 0.5 means partial vaccination reduces the infection risk by 50%.
    "full_vaccine_effectiveness": 0,  # Effectiveness of full vaccination (two or more doses) in reducing infection probability.
    # The infection rate is multiplied by this factor for fully vaccinated individuals.
    # A value of 0 implies complete immunity, with no risk of infection after full vaccination.
    "mask_policy": MASK_POLICY,  # Function that defines the mask-wearing policy for individuals.
    # This function is invoked daily to decide whether each individual should wear a mask.
    "dist_policy": SOCIAL_DIST_POLICY,  # Function that defines the social distancing policy for individuals.
    # This function is invoked daily to determine each individual’s social distancing behavior.
    "vac_policy": VAC_POLICY,  # Function that defines the vaccination policy for the simulation.
    # This function is called daily to decide which individuals receive vaccine doses.
    "random_seed": 42,  # Seed for the random number generator, ensuring reproducibility of the simulation.
    # Setting this seed allows the simulation to produce the same results on repeated runs.
}

# Number of simulation runs
replications = 100

# Create the SimulationBatchRunner
batch_runner = SimulationBatchRunner(
    replications=replications,
    simulation_params=simulation_params,
)

# Run all simulations
batch_runner.run()

# Define the path to save the results
hist_save_path = os.path.join(
    results_dir,
    f"ir{0.4}_ms{9999}_sds{9999}_hist.png",
)

state_save_path = os.path.join(
    results_dir,
    f"ir{0.4}_ms{9999}_sds{9999}_state.png",
)

# Titles for the plots
title_hist = (
    f"Statistical Summary\n"
    f"Infection Rate: {0.4}\n"
    f"Mask Start Day: None\n"
    f"Social Distancing Start Day: None"
)

title_state = (
    f"State Over Time\n"
    f"Infection Rate: {0.4}\n"
    f"Mask Start Day: None\n"
    f"Social Distancing Start Day: None"
)

# Plot histograms with statistical summaries and save results
batch_runner.plot_histograms(
    save_path=hist_save_path,
    print_graphs=False,
    title=title_hist,
)


# Plot the expected counts over time and save results
batch_runner.plot_state_over_time(
    save_path=state_save_path,
    print_graphs=False,
    title=title_state,
)


INFO:src.batch_run:Starting batch run of simulations.
INFO:src.batch_run:All simulations completed. Processing results...
INFO:src.batch_run:Results aggregation completed.


Histogram saved to c:\Users\louis\OneDrive - Georgia Institute of Technology\GATech\2024 sem 4 fall\ISYE 6644\Project\pandemic-flu-spread\pandemic-flu-spread\results\ir0.4_ms9999_sds9999_hist.png
Plot saved to c:\Users\louis\OneDrive - Georgia Institute of Technology\GATech\2024 sem 4 fall\ISYE 6644\Project\pandemic-flu-spread\pandemic-flu-spread\results\ir0.4_ms9999_sds9999_state.png


,State,Day,Mean,Std Dev
0,susceptible,0,9900.0,0.0
1,susceptible,1,9900.0,0.0
2,susceptible,2,9900.0,0.0
3,susceptible,3,9900.0,0.0
4,susceptible,4,9900.0,0.0
...,...,...,...,...
489,vaccine_supply,33,0.0,0.0
490,vaccine_supply,34,0.0,0.0
491,vaccine_supply,35,0.0,0.0
492,vaccine_supply,36,0.0,0.0
